Generate defect configurations, reasonable ones.

In [31]:
from pprint import pprint
from pymatgen.analysis.local_env import CrystalNN
from pymatgen.analysis.structure_prediction.dopant_predictor import (
    get_dopants_from_shannon_radii,
    get_dopants_from_substitution_probabilities
)
from pymatgen.ext.matproj import MPRester

api_key = 'Yd2jih656GIoA2Ksfu57XrmOGh6seR2P'  # INSERT YOUR OWN API KEY
mpr = MPRester(api_key=api_key)
num_dopants = 5

In [32]:
from pymatgen.io.vasp import Poscar
import json

with open('SM_dataset/Te3Y2_653174.json', 'r') as file:
    data = json.load(file)
    structure_string = data["Structure_rlx"]

# Preprocess the string to remove the first line
structure_string = '\n'+structure_string.split('\n', 1)[1]

# Use pymatgen to parse the structure
poscar = Poscar.from_str(structure_string, read_velocities=False)
# poscar.write_file('./POSCAR')
structure = poscar.structure
print(sum(poscar.natoms))

# Add oxidation states (optional)
structure.add_oxidation_state_by_guess()

# Access species
species = structure.composition.elements

print("Species in structure:", species)

20
Species in structure: [Species Y3+, Species Te2-]


/home/vipandyc/miniconda3/envs/cava/lib/python3.10/site-packages/pymatgen/io/vasp/inputs.py:462: BadPoscarWarning: Ignoring selective dynamics tag, as no ionic degrees of freedom were fixed.
  warnings.warn(


In [33]:
cnn = CrystalNN()
bonded_structure = cnn.get_bonded_structure(structure)

dopants = get_dopants_from_shannon_radii(bonded_structure, num_dopants=num_dopants)

pprint(dopants)

{'n_type': [{'dopant_species': Species Ce4+,
             'original_species': Species Y3+,
             'radii_diff': -0.030000000000000027},
            {'dopant_species': Species Po4+,
             'original_species': Species Y3+,
             'radii_diff': 0.039999999999999925},
            {'dopant_species': Species Th4+,
             'original_species': Species Y3+,
             'radii_diff': 0.039999999999999925},
            {'dopant_species': Species Pu4+,
             'original_species': Species Y3+,
             'radii_diff': -0.040000000000000036},
            {'dopant_species': Species I5+,
             'original_species': Species Y3+,
             'radii_diff': 0.04999999999999993}],
 'p_type': [{'dopant_species': Species Pd2+,
             'original_species': Species Y3+,
             'radii_diff': -0.040000000000000036},
            {'dopant_species': Species Cd2+,
             'original_species': Species Y3+,
             'radii_diff': 0.04999999999999993},
            

/home/vipandyc/miniconda3/envs/cava/lib/python3.10/site-packages/pymatgen/analysis/structure_prediction/dopant_predictor.py:104: UserWarning: Shannon radius not found for Te2- with coordination number 4.
Skipping...
  warnings.warn(f"Shannon radius not found for {species} with coordination number {cn}.\nSkipping...")


In [34]:
threshold = 0.001  # probability threshold for substitution/structure predictions
dopants = get_dopants_from_substitution_probabilities(
    structure, num_dopants=num_dopants, threshold=threshold
)

#pprint(dopants)
dopants

ValueError: the species Te2- is not allowed for the probability model you are using